In [ ]:
import torch
import numpy
import time

from torch.utils.data import DataLoader

from utils.model_util import *
from utils.data_util import *
from utils.lib_util import *
from utils.train_util import *
from trainer import *

In [ ]:
args = get_args()
all_server = [i for i in range(9)]
num_target = 10
num_server_client = 3
train_dataset_o, test_dataset_o, c, h, w = get_dataset(args.dataset)
client_model, server_model = intial_model(
    args, num_target, num_server_client, c, h, w)
target_list = {0: [0, 1, 2], 1: [3, 4, 5], 2: [6, 7, 8, 9]}
[public_dataset, test_dataset] = split_parts_random(
    test_dataset_o, [args.num_public_data, int(len(test_dataset_o)) - args.num_public_data])
public_dataloader = DataLoader(
    dataset=public_dataset,
    batch_size=args.batch_size,
    shuffle=True,
    pin_memory=True,
    num_workers=args.num_workers)
keys = ['server_model',
        'train_dataloader',
        'test_dataloader',
        'validate_dataloader',
        'public_dataloader',
        'log',
        'num_target',
        'client_accuracy',
        'client_loss',
        'validate_accuracy',
        'weight_list',
        'weight_server',
        'server_client',
        'all_server',
        'client_model',
        'target_list',
        'public_dataset']
values = [server_model,
          train_dataloader,
          test_dataloader,
          validate_dataloader,
          public_dataloader,
          log,
          num_target,
          client_accuracy,
          client_loss,
          validate_accuracy,
          weight_list,
          weight_server,
          server_client,
          all_server,
          client_model,
          target_list,
          public_dataset]
args_train = dict(zip(keys, values))

In [ ]:
def distill_public(args, args_train):
    server_model = {}
    for server in range(9):
        server_model = server_model[server].eval()
    dataset_ = []
    target_logits = {}
    for i, (data, target) in enumerate(args_train['public_dataset']):
        data_device = data.to(args.device)
        data_ = torch.unsqueeze(data_device, dim=0)
        logits = torch.zeros([1, 10]).to(args.device)
        num_target_servers = 0
        for server in args_train['all_server']:
            if target in args_train['target_list'][server]:
                logits += server_model(data_)
                num_target_servers += 1
        logits = logits / num_target_servers
        dataset_.append((data, i))
        target_logits[i] = (target, logits)
    return dataset_, target_logits